In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import math
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
# data_dir = pathlib.Path(archive).with_suffix('')

In [ ]:
# image_count = len(list(data_dir.glob('*/*.jpg')))
# print(image_count)

In [2]:
data_dir = "C:/Users/welcome/.keras/datasets/flower_photos/"

In [3]:
batch_size = 32
img_height = 180
img_width = 180

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [ ]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")

In [7]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 180, 180, 3)
(32,)


In [8]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [9]:
normalized_ds_train = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds_train))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [10]:
normalized_ds_val = val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds_val))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.94418466


In [11]:
vgg19 = tf.keras.applications.vgg19.VGG19(
    include_top = False, 
    input_shape = (img_width, img_height, 3), 
    weights = 'imagenet')
# vgg19.summary()  ### To show the model's architecture

In [12]:
model = Sequential()
for layer in vgg19.layers:
    layer.trainable = False #o turn off VGG19's trainable weights
model.add(vgg19)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 5, 5, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 1024)              13108224  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 256)               262400    
                                                                 
 dense_2 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 33396293 (127.40 MB)
Trainable params: 133

In [13]:
# To select the optimizer ans the loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [14]:
# Compile the model
model.compile(optimizer=optimizer, loss=loss_object, metrics=['accuracy'])

In [15]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [16]:
batch_size = 64
# Calculate the number of batches per epoch
import math
n_batches = 2936 / batch_size
n_batches = math.ceil(n_batches)


In [17]:
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
#     save_best_only=True,
    verbose=1,
    mode='auto',
    save_weights_only=True,
    period=1,
    save_freq=1*n_batches
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)
callbacks = [checkpoint, earlystop]

### After epoch = 1, I stopped the training process 

In [18]:
# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

### To fit the model
history = model.fit(
    normalized_ds_train, 
    epochs = 3,
    validation_data = normalized_ds_val, 
    callbacks = callbacks,
    shuffle = True, 
#     verbose=0
)

Epoch 1/3


D:\github\tf\lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


45/92 [=============>................] - ETA: 1:32 - loss: 1.6071 - accuracy: 0.5118
Epoch 1: saving model to training\cp-0001.ckpt
91/92 [============================>.] - ETA: 1s - loss: 1.2091 - accuracy: 0.5982
Epoch 1: saving model to training\cp-0001.ckpt
92/92 [==============================] - 224s 2s/step - loss: 1.2074 - accuracy: 0.5984 - val_loss: 0.7649 - val_accuracy: 0.7316
Epoch 2/3
 7/92 [=>............................] - ETA: 2:38 - loss: 0.5767 - accuracy: 0.7991

KeyboardInterrupt: 

In [19]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training\\cp-0001.ckpt'

In [20]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

### Rebuilding architecture

In [21]:
vgg19 = tf.keras.applications.vgg19.VGG19(
    include_top = False, 
    input_shape = (img_width, img_height, 3), 
    weights = 'imagenet')
# vgg19.summary()  ### To show the model's architecture

In [22]:
model = Sequential()
for layer in vgg19.layers:
    layer.trainable = False #o turn off VGG19's trainable weights
model.add(vgg19)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 5, 5, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 12800)             0         
                                                                 
 dense_3 (Dense)             (None, 1024)              13108224  
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               262400    
                                                                 
 dense_5 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 33396293 (127.40 MB)
Trainable params: 1

In [23]:
# To select the optimizer ans the loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [24]:
# Compile the model
model.compile(optimizer=optimizer, loss=loss_object, metrics=['accuracy'])

In [25]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [26]:
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
#     save_best_only=True,
    verbose=1,
    mode='auto',
    save_weights_only=True,
    period=1,
    save_freq=1*n_batches
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)
callbacks = [checkpoint, earlystop]

In [27]:
# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

In [28]:
# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

### To fit the model
history = model.fit(
    normalized_ds_train, 
    epochs = 3,
    validation_data = normalized_ds_val, 
    callbacks = callbacks,
    shuffle = True, 
#     verbose=0
)

Epoch 1/3
45/92 [=============>................] - ETA: 1:32 - loss: 0.5584 - accuracy: 0.8111
Epoch 1: saving model to checkpoints\cp-0001.ckpt
91/92 [============================>.] - ETA: 1s - loss: 0.5809 - accuracy: 0.7970
Epoch 1: saving model to checkpoints\cp-0001.ckpt
92/92 [==============================] - 227s 2s/step - loss: 0.5792 - accuracy: 0.7977 - val_loss: 0.6517 - val_accuracy: 0.7657
Epoch 2/3
45/92 [=============>................] - ETA: 1:30 - loss: 0.4214 - accuracy: 0.8410
Epoch 2: saving model to checkpoints\cp-0002.ckpt
91/92 [============================>.] - ETA: 1s - loss: 0.4347 - accuracy: 0.8372
Epoch 2: saving model to checkpoints\cp-0002.ckpt
92/92 [==============================] - 220s 2s/step - loss: 0.4337 - accuracy: 0.8375 - val_loss: 0.6894 - val_accuracy: 0.7779
Epoch 3/3
45/92 [=============>................] - ETA: 1:33 - loss: 0.3062 - accuracy: 0.8861
Epoch 3: saving model to checkpoints\cp-0003.ckpt
91/92 [============================>.] 

In [29]:
# Evaluate the model
loss, acc = model.evaluate(normalized_ds_val)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

23/23 [==============================] - 41s 2s/step - loss: 0.8124 - accuracy: 0.7616
Restored model, accuracy: 76.16%
